This notebook can be used to create exponential smoothing estimates for the next year. This is a very simple technique for predicting the next value in a time series. I experimented with a few other methods like k nearest neighbors and arima models but they all probably need more data with a more discernable pattern for decent predictions.

Exponential smoothing tends to give predictions that are fairly close to the last few years and so its predictions tend to be anchored to current values and predicted increase/decrease tends to be small. I think this approach is warr

I am using these exponential smoothing predictions to update the dashboard annotations with an estimate of whether the next year is expected to remain stable, increase, or decrease. I think this is a reasonable use of these predictions considering that they aren't very precise and the underlying structure of the data is marred by collection inconsistency.

Run this notebook to update the blurbs.csv with Percent Change Predicted values which will be read by the dashboard.

In [16]:
import pandas as pd
import os
import numpy as np



def clean_data(totals_data_path, per_student_data_path, main_unit):
    # Load the data
    totals_data = pd.read_csv(totals_data_path)
    per_student_data = pd.read_csv(per_student_data_path)

    # Convert 'Scope' to string
    totals_data['Scope'] = totals_data['Scope'].astype(str)
    per_student_data['Scope'] = per_student_data['Scope'].astype(str)

    # Select relevant columns
    totals_data = totals_data[['Fiscal Year', 'Scope', 'Source', main_unit]]
    per_student_data = per_student_data[['Fiscal Year', 'Scope', 'Source', main_unit]]

    # Convert main_unit column to float
    # Check if main_unit column is a string, if so remove commas and convert to float
    if totals_data[main_unit].dtype == 'object':
        totals_data[main_unit] = totals_data[main_unit].str.replace(',', '').astype(float)
    if per_student_data[main_unit].dtype == 'object':
        per_student_data[main_unit] = per_student_data[main_unit].str.replace(',', '').astype(float)

    # Combine sources
    def combine_sources(source):
        if isinstance(source, str):
            if 'commuting' in source.lower():
                return 'Commuting'
            elif 'co-gen' in source.lower():
                return 'Co-gen Plant'
            else:
                return source
        else:
            return source

    totals_data['Source'] = totals_data['Source'].apply(combine_sources)
    per_student_data['Source'] = per_student_data['Source'].apply(combine_sources)

    # Sum up the 'Commuting' rows for each year
    totals_data = totals_data.groupby(['Fiscal Year', 'Scope', 'Source'], as_index=False)[main_unit].sum()
    per_student_data = per_student_data.groupby(['Fiscal Year', 'Scope', 'Source'], as_index=False)[main_unit].sum()

    # Replace source names in both dataframes
    replacements = {
        'Fertilizer & Animals': 'Fertilizer',
        'Co-gen steam': 'Co-gen Plant',
        'Direct Transportation': 'University Fleet',
        'Other On-Campus Stationary': 'Propane & Natural Gas',
        'Directly Financed Air Travel': 'Air Travel',
        'Solid Waste': 'Landfill Waste',
        'Other Directly Financed Travel': 'Bus Travel'
    }

    totals_data['Source'] = totals_data['Source'].replace(replacements)
    per_student_data['Source'] = per_student_data['Source'].replace(replacements)

    return totals_data, per_student_data, main_unit

cleaned_totals, cleaned_per_student, main_unit = clean_data('total.csv', 'per_student.csv', 'GHG MTCDE')

In [17]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def predict_next_year_exp_smoothing(df, year):
    # Get the unique sources
    sources = df['Source'].unique()

    # Initialize list to store predictions
    predictions = []

    # For each source, calculate the exponential smoothing of the last few years
    for source in sources:
        source_data = df[df['Source'] == source]

        # Fit exponential smoothing model
        model = ExponentialSmoothing(source_data['GHG MTCDE'], trend='add')
        model_fit = model.fit()

        # Predict next year
        next_year_prediction = model_fit.predict(len(source_data), len(source_data))

        # Append prediction to list
        predictions.append({
            'Fiscal Year': year,
            'Scope': source_data['Scope'].iloc[0],  # Assuming Scope doesn't change over years
            'Source': source,
            'GHG MTCDE': next_year_prediction[0]
        })

    # Convert list of predictions to DataFrame
    predictions_df = pd.DataFrame(predictions)

    return predictions_df

# Predict for the next year
next_year_predictions = predict_next_year_weighted(cleaned_totals, 2024)

In [18]:
# Get the data for 2023
data_2023 = cleaned_totals[cleaned_totals['Fiscal Year'] == 2023]

# Merge the data for 2023 and the predictions for 2024
merged_data = pd.merge(data_2023, next_year_predictions, on='Source', suffixes=('_2023', '_2024'))

# Calculate the percent change from 2023 to 2024
merged_data['Percent Change'] = ((merged_data['GHG MTCDE_2024'] - merged_data['GHG MTCDE_2023']) / merged_data['GHG MTCDE_2023']) * 100

# Print the percent change
merged_data[['Source', 'Percent Change']]





,Source,Percent Change
0,Co-gen Plant,0.901913
1,University Fleet,-4.577733
2,Fertilizer,-15.376344
3,Propane & Natural Gas,-3.681468
4,Purchased Electricity,-0.296090
5,Commuting,0.421106
6,Air Travel,-10.113273
7,FERA,0.901932
8,Bus Travel,-4.457364
9,Landfill Waste,-1.912545


In [19]:
import pandas as pd

# Load blurbs from csv
blurbs = pd.read_csv('blurbs.csv')

# Merge the blurbs data with the percent change data
merged_blurbs = pd.merge(blurbs, merged_data[['Source', 'Percent Change']], on='Source')

# Update the 'Percent Change Predicted' column in the blurbs data
blurbs['Percent Change Predicted'] = merged_blurbs['Percent Change']

# Save the updated blurbs data back to csv
blurbs.to_csv('blurbs.csv', index=False)


In [20]:
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets

# Create a dropdown menu for source selection
source_dropdown = widgets.Dropdown(
    options=cleaned_totals['Source'].unique().tolist(),
    value=cleaned_totals['Source'].unique().tolist()[0],
    description='Source:',
)

# Function to update the plot based on selected source
def update_plot(source):
    # Filter data for the selected source
    source_data = cleaned_totals[cleaned_totals['Source'] == source]
    prediction_data = next_year_predictions[next_year_predictions['Source'] == source]

    # Combine historical and predicted data
    combined_data = pd.concat([source_data, prediction_data])

    # Create the plot
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=combined_data, x='Fiscal Year', y='GHG MTCDE')
    plt.title(f'GHG Emissions for {source} (Including 2024 Prediction)')

# Display the dropdown menu and update the plot when a selection is made
_ = widgets.interact(update_plot, source=source_dropdown)
_ = None


interactive(children=(Dropdown(description='Source:', options=('Co-gen Plant', 'University Fleet', 'Fertilizer…